In [1]:
import pandas as pd
import nlp_util
from termcolor import colored


/Users/kent/git/kent-ai-learning-notebook/venv/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_csv('./train.csv')
df[['POI', 'street']] = df['POI/street'].str.split('/', 1, expand=True)
df = df.drop(labels=['POI/street'], axis=1)

In [3]:
df.head(2)

,id,raw_address,POI,street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,,jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",,


In [4]:
poi,street,raw = "toko bb kids", "raya samb gede", "xxx raya. sa-mb gede, 299 toko bb k&ids yyy",
p_start,p_end,s_start,s_end = nlp_util.get_bio_tagging_range(raw,street,poi)

In [5]:
df_poi = df[df['POI'] != '']

In [25]:
pair_list = []
for id,row in df_poi.iterrows():
    
    street_str = row.street
    POI_str = row.POI
    raw_str = row.raw_address
    
    try:
    
        p_start,p_end,s_start,s_end = nlp_util.get_bio_tagging_range(raw_str,street_str,POI_str)
    #     print(POI_str,"===>",raw_str[:p_start]+ colored(raw_str[p_start:p_end],"red")+raw_str[p_end:])

        raw = nlp_util.prepare_text(raw_str[p_start:p_end])
        label = nlp_util.prepare_text(POI_str)

    #     print(target,label)

        pair_result = nlp_util.get_fuzzy_pairs(raw, label) 

        pair_list.append(pair_result)
    except:
        print("*",end='')


**************************************************************

In [31]:
pair_list[22]

[0, [('rnd', 'rnd'), ('printing', 'prin')]]

In [43]:
from collections import Counter
poi_s2l = {}
for _ , pairs in pair_list:
    for long_str,short_str in pairs:
        if short_str == None: continue

        if len(long_str) > len(short_str) :
            
            if short_str in poi_s2l:
                cc = poi_s2l[short_str]
                cc.update([long_str])
            else:
                poi_s2l[short_str] = Counter([long_str])
        
    

In [46]:
import pickle
with open("./poi_s2l.pkl","wb") as fw:
    pickle.dump(poi_s2l,fw)
